In [43]:
import pandas as pd
import re

from tqdm.notebook import tqdm
tqdm.pandas()

In [23]:
abbr_term = pd.read_csv('../abbr_parsing/abbr_term.csv', sep='\t')

In [24]:
records_df = pd.read_csv('../data/I10_anamnesis.csv', sep='\t',)

In [25]:
def extract_term_context(term, text, window=5):
    # term = term.lower()
    # text = text.lower()
    
    term_splited = term.split()
    term_length = len(term_splited)
    
    context_list = []
    
    words = text.split()
    
    context_size = window*2 + term_length + term_length % 2
    current_context = [word for word in words[:context_size]]

    if all(word in context_list for word in term.split()):
        context_list.append(" ".join(current_context))
    
    for word in words[context_size:]:
        term_slice = slice(window - term_length // 2, window + term_length // 2 + int(term_length % 2 != 0))
        
        if current_context[term_slice] == term_splited:
            context_list.append(" ".join(current_context))
        
        current_context.append(word)
        del current_context[0]
    
    if all(word in context_list for word in term.split()):
        context_list.append(" ".join(current_context))
    
    return context_list

In [26]:
all_words = frozenset(word.lower() for text in records_df['RecordEMC'] for word in text.split())

In [ ]:
term_df = pd.DataFrame()

for term in tqdm(abbr_term.term.dropna()):
    if all(word in all_words for word in term.split()):
        context_list = records_df.RecordEMC.apply(lambda text: extract_term_context(term, text))
        event_context = [
            (event, context) 
            for event, context in zip(records_df.MedicalProcessEventCode, context_list)
            if len(context) > 0
        ]
        
        term_df = term_df.append(pd.DataFrame({
            'term': [term] * len(event_context),
            'event_code': [event for event, _ in event_context],
            'context': [context for _, context in event_context],
        }))
        
term_df = term_df.reset_index(drop=True)

  0%|          | 0/7242 [00:00<?, ?it/s]

In [52]:
term_df.shape

(0, 3)

In [63]:
term_context = term_df.explode('context')

In [80]:
abbr_term.head()

,name,mkb_code,start_year,age_category,end_year,id,abbreviation,term
0,Рак печени (гепатоцеллюлярный),C22.0,2020,Взрослые,2022.0,1,АЛТ,аланинаминотрансфераза
1,Рак печени (гепатоцеллюлярный),C22.0,2020,Взрослые,2022.0,1,АСТ,аспартатаминотрансфераза
2,Рак печени (гепатоцеллюлярный),C22.0,2020,Взрослые,2022.0,1,АФП,альфа-фетопротеин
3,Рак печени (гепатоцеллюлярный),C22.0,2020,Взрослые,2022.0,1,ГЦР,гепатоцеллюлярный рак
4,Рак печени (гепатоцеллюлярный),C22.0,2020,Взрослые,2022.0,1,КТ,компьютерная томография


In [83]:
term2abbr = {
    term: abbr for term, abbr in abbr_term[['term', 'abbreviation']].iloc
}

In [85]:
term_context['mcb_code'] = ['I10'] * term_context.shape[0]
term_context['abbreviation'] = term_context.term.map(term2abbr)

In [93]:
term_context = term_context.drop_duplicates('context')

In [94]:
term_context[['event_code', 'mcb_code', 'abbreviation', 'term', 'context']].to_csv('term_context.csv', sep='\t', index=False)

- Долю полных терминов vs аббревиатур